# Regression

In [1]:
import re
from functools import partial
from typing import List, Tuple

In [2]:
import sys
sys.path.append('lib')

In [3]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import patsy

In [4]:
import nsfg
import fwf

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
from IPython.core.pylabtools import figsize
sns.set_theme()
figsize(11, 5)

In [7]:
r4 = partial(np.round, decimals=4)

## StatsModels

Let's load up the NSFG data again.

In [8]:
live = nsfg.read_live_fem_preg()

In [9]:
live.loc[:, ['totalwgt_lb', 'agepreg']].apply(lambda col: col.isna().sum())

totalwgt_lb    0
agepreg        0
dtype: int64

Here's birth weight as a function of mother's age (which we saw in the previous chapter).

In [10]:
model = smf.ols('totalwgt_lb ~ agepreg', data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     43.02
Date:                Fri, 21 Jan 2022   Prob (F-statistic):           5.72e-11
Time:                        23:48:32   Log-Likelihood:                -15897.
No. Observations:                9038   AIC:                         3.180e+04
Df Residuals:                    9036   BIC:                         3.181e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.8304      0.068    100.470      0.000       6.697       6.964
agepreg        0.0175      0.003      6.559      0.000       0.012       0.023
==============================================================================
Omnibus:                     1024.052   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3081.833
Skew:                          -0.601   Prob(JB):                         0.00
Kurtosis:                       5.596   Cond. No.                         118.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We can extract the parameters, and the p-value of the slope estimate.

In [89]:
def summarize_results(results):
    """Prints the most important parts of linear regression results:

    results: RegressionResults object
    """
    for name, param in results.params.items():
        pvalue = results.pvalues[name]
        print(f'{name:26}: {param:0.4f}: {pvalue:0.4f}')
    try:
        print(f'R^2      : {results.rsquared:0.4f}')
        print(f'Std(ys)  : {results.model.endog.std():0.4f}')
        print(f'Std(res) : {results.resid.std():0.4f}')
    except AttributeError:
        print(f'R^2      : {results.prsquared:0.4f}')

In [12]:
summarize_results(results)

Intercept           : 6.8304    0.0000
agepreg             : 0.0175    0.0000
R^2      : 0.0047
Std(ys)  : 1.4082
Std(res) : 1.4050


In [13]:
inter = results.params['Intercept']
slope = results.params['agepreg']
slope_pvalue = results.pvalues['agepreg']
r4((inter, slope, slope_pvalue))

array([6.8304, 0.0175, 0.    ])

And the coefficient of determination.

In [14]:
r4(results.rsquared)

0.0047

The `std` of the dependent variable and the residuals

In [15]:
r4(live.totalwgt_lb.std())

1.4083

In [16]:
r4(results.resid.std())

1.405

`std(ys)` is the standard deviation of the dependent variable, which is the RMSE if you have to guess birth weights without the benefit of any explanatory variables. `std(res)` is the standard deviation of the residuals, which is the RMSE if your guesses are informed by the mother’s age. As we have already seen, knowing the mother’s age provides no substantial improvement to the predictions.

## Multiple regression

In [Chapter 4](04_Cumulative_Distribution_Functions.ipynb) we saw that first babies tend to be lighter than others, and this effect is statistically significant. But it is a strange result because there is no obvious mechanism that would cause first babies to be lighter. So we might wonder whether this relationship is spurious.

In fact, there is a possible explanation for this effect. We have seen that birth weight depends on mother’s age, and we might expect that mothers of first babies are younger than others.

With a few calculations we can check whether this explanation is plausible. Then we’ll use multiple regression to investigate more carefully. First, let’s see how big the difference in weight is:

In [17]:
results = smf.ols('totalwgt_lb ~ agepreg', data=live).fit()
slope = results.params['agepreg']

In [18]:
live.loc[:, ['birthcat', 'totalwgt_lb', 'agepreg']].groupby('birthcat').mean()

,totalwgt_lb,agepreg
birthcat,,
firsts,7.201094,23.095343
others,7.325856,26.654064


In [19]:
diff_weight = np.diff(live.groupby('birthcat')['totalwgt_lb'].mean()).item()
r4(diff_weight)

0.1248

First babies are 0.125 lbs lighter, or 2 ounces. And the difference in ages:

In [20]:
diff_age = np.diff(live.groupby('birthcat')['agepreg'].mean()).item()
r4(diff_age)

3.5587

The mothers of first babies are 3.59 years younger. Running the linear model again, we get the change in birth weight as a function of age:

The slope is 0.0175 pounds per year. If we multiply the slope by the difference in ages, we get the expected difference in birth weight for first babies and others, due to mother’s age:

In [21]:
r4(slope * diff_age)

0.0621

The result is 0.063, just about half of the observed difference. So we conclude, tentatively, that the observed difference in birth weight can be partly explained by the difference in mother’s age.

Using multiple regression, we can explore these relationships more systematically.

In [22]:
live['isfirst'] = live.birthcat == 'firsts' 
results = smf.ols('totalwgt_lb ~ isfirst', data=live).fit()
summarize_results(results)

Intercept           : 7.3259    0.0000
isfirst[T.True]     : -0.1248    0.0000
R^2      : 0.0020
Std(ys)  : 1.4082
Std(res) : 1.4069


Because `isfirst` is a boolean, ols treats it as a categorical variable, which means that the values fall into categories, like True and False, and should not be treated as numbers. The estimated parameter is the effect on birth weight when isfirst is true, so the result, -0.125 lbs, is the difference in birth weight between first babies and others.

The slope and the intercept are statistically significant, which means that they were unlikely to occur by chance, but the the $R^2$ value for this model is small, which means that `isfirst` doesn’t account for a substantial part of the variation in birth weight.

The results are similar with agepreg:

In [23]:
results = smf.ols('totalwgt_lb ~ agepreg', data=live).fit()
summarize_results(results)

Intercept           : 6.8304    0.0000
agepreg             : 0.0175    0.0000
R^2      : 0.0047
Std(ys)  : 1.4082
Std(res) : 1.4050


Again, the parameters are statistically significant, but $R^2$ is low.

These models confirm results we have already seen. But now we can fit a single model that includes both variables

In [24]:
results = smf.ols('totalwgt_lb ~ isfirst + agepreg', data=live).fit()
summarize_results(results)

Intercept           : 6.9142    0.0000
isfirst[T.True]     : -0.0698    0.0253
agepreg             : 0.0154    0.0000
R^2      : 0.0053
Std(ys)  : 1.4082
Std(res) : 1.4046


As expected, when we control for mother's age, the apparent difference due to `isfirst` is cut in half.

## Nonlinear relationships

Remembering that the contribution of agepreg might be nonlinear, we might consider adding a variable to capture more of this relationship. One option is to create a column, `agepreg2`, that contains the squares of the ages:

In [25]:
live['agepreg2'] = live.agepreg**2
results = smf.ols('totalwgt_lb ~ isfirst + agepreg + agepreg2', data=live).fit()
summarize_results(results)

Intercept           : 5.6923    0.0000
isfirst[T.True]     : -0.0504    0.1091
agepreg             : 0.1124    0.0000
agepreg2            : -0.0018    0.0000
R^2      : 0.0075
Std(ys)  : 1.4082
Std(res) : 1.4030


Now by estimating parameters for agepreg and agepreg2, we are effectively fitting a parabola

The parameter of `agepreg2` is negative, so the parabola curves downward, which is consistent with the shape of the lines in chapter 10. The quadratic model of `agepreg` accounts for more of the variability in birth weight; the parameter for isfirst is smaller in this model, and no longer statistically significant.

Using computed variables like `agepreg2` is a common way to fit polynomials and other functions to data. This process is still considered linear regression, because the dependent variable is a linear function of the explanatory variables, regardless of whether some variables are nonlinear functions of others.

When we do that, the apparent effect of `isfirst` gets even smaller, and is no longer statistically significant.

These results suggest that the apparent difference in weight between first babies and others might be explained by difference in mothers' ages, at least in part.

In this example, mother’s age acts as a control variable; including agepreg in the model “controls for” the difference in age between first-time mothers and others, making it possible to isolate the effect (if any) of isfirst.

## Data Mining

Now suppose that you really want to win the pool. What could you do to improve your chances? Well, the NSFG dataset includes 244 variables about each pregnancy and another 3087 variables about each respondent. Maybe some of those variables have predictive power. To find out which ones are most useful, why not try them all?

Testing the variables in the pregnancy table is easy, but in order to use the variables in the respondent table, we have to match up each pregnancy with a respondent. In theory we could iterate through the rows of the pregnancy table, use the caseid to find the corresponding respondent, and copy the values from the correspondent table into the pregnancy table. But that would be slow.

We can use `join` to combine variables from the preganancy and respondent tables.

In [26]:
live.query('prglngth > 30', inplace=True)

In [27]:
resp = nsfg.read_fem_resp().set_index('caseid')

In [28]:
# suffix appended to overlapping columns in the right table
join = live.join(resp, on='caseid', rsuffix='_r')

And we can search for variables with explanatory power.

Because we don't clean most of the variables, we are probably missing some good ones.

In [29]:
def go_mining(df: pd.DataFrame):
    """Searches for variables that predict birth weight.

    df: DataFrame of pregnancy records

    returns: list of (rsquared, variable name) pairs
    """
    variables = []
    for name in df.columns:
        try:
            # check that the explanatory variable has some variability
            if df[name].var() < 1e-7:
                continue
            formula = 'totalwgt_lb ~ agepreg + ' + name
            model = smf.ols(formula, data=df)
            # reject models that use less than half of the data
            if model.nobs < len(df)/2:
                continue
            results = model.fit()
            variables.append((round(results.rsquared, 4), name))
        except (ValueError, TypeError, patsy.PatsyError) as e:
            continue
    return variables

For each variable we construct a model, compute $R^2$ , and append the results to a list. The models all include agepreg, since we already know that it has some predictive power.

I check that each explanatory variable has some variability; otherwise the results of the regression are unreliable. I also check the number of observations for each model. Variables that contain a large number of nans are not good candidates for prediction.

For most of these variables, we haven’t done any cleaning. Some of them are encoded in ways that don’t work very well for linear regression. As a result, we might overlook some variables that would be useful if they were cleaned properly. But maybe we will find some good candidates.

In [30]:
variables = go_mining(join)

In [31]:
variables.sort(reverse=True)

In [32]:
variables[:30]

[(1.0, 'totalwgt_lb'),
 (0.9498, 'birthwgt_lb'),
 (0.3008, 'lbw1'),
 (0.1301, 'prglngth'),
 (0.1234, 'wksgest'),
 (0.102, 'agecon'),
 (0.0271, 'mosgest'),
 (0.0186, 'babysex'),
 (0.0162, 'race_r'),
 (0.0162, 'race'),
 (0.016, 'nbrnaliv'),
 (0.014, 'paydu'),
 (0.0134, 'rmarout03'),
 (0.0131, 'birthwgt_oz'),
 (0.0125, 'anynurse'),
 (0.0122, 'bfeedwks'),
 (0.0119, 'totincr'),
 (0.0118, 'marout03'),
 (0.0118, 'marcon03'),
 (0.0114, 'rmarout6'),
 (0.0114, 'rmarout01'),
 (0.0114, 'cebow'),
 (0.0113, 'marout01'),
 (0.0112, 'hisprace_r'),
 (0.0112, 'hisprace'),
 (0.011, 'mar1diss'),
 (0.0106, 'fmarcon5'),
 (0.0105, 'rmarout02'),
 (0.0105, 'marcon02'),
 (0.0105, 'fmarout5')]

The following functions report the variables with the highest values of $R^2$.

In [33]:
def read_variables():
    vars = fwf.read_stata_dictionary('data/2002FemPreg.dct')
    vars.extend(fwf.read_stata_dictionary('data/2002FemResp.dct'))
    return vars

Some of the variables that do well are not useful for prediction because they are not known ahead of time.

Combining the variables that seem to have the most explanatory power.

In [34]:
# try adding lbw1
formula = ('totalwgt_lb ~ agepreg + C(race) + babysex==1 + '
               'nbrnaliv>1 + paydu==1 + totincr')
results = smf.ols(formula, data=join).fit()
summarize_results(results)

Intercept           : 6.6303    0.0000
C(race)[T.2]        : 0.3570    0.0000
C(race)[T.3]        : 0.2665    0.0000
babysex == 1[T.True]: 0.2952    0.0000
nbrnaliv > 1[T.True]: -1.3783    0.0000
paydu == 1[T.True]  : 0.1196    0.0001
agepreg             : 0.0074    0.0035
totincr             : 0.0122    0.0019
R^2      : 0.0600
Std(ys)  : 1.2711
Std(res) : 1.2325


## Logistic regression

As an example of logistic regression, suppose a friend of yours is pregnant and you want to predict whether the baby is a boy or a girl. You could use data from the NSFG to find factors that affect the “sex ratio”, which is conventionally defined to be the probability of having a boy.

Example: suppose we are trying to predict `y` using explanatory variables `x1` and `x2`.

In [35]:
y = np.array([0, 1, 0, 1])
# think of these as feature column vectors
x1 = np.array([0, 0, 0, 1])
x2 = np.array([0, 1, 1, 1])

According to the logit model the log odds for the $i$th element of $y$ is

$\log o = \beta_0 + \beta_1 x_1 + \beta_2 x_2 $

So let's start with an arbitrary guess about the elements of $\beta$:



In [36]:
beta = [-1.5, 2.8, 1.1]

Plugging in the model, we get log odds.

In [37]:
log_o = beta[0] + beta[1] * x1 + beta[2] * x2
log_o

array([-1.5, -0.4, -0.4,  2.4])

Which we can convert to odds.

In [38]:
o = np.exp(log_o)
o

array([ 0.22313016,  0.67032005,  0.67032005, 11.02317638])

And then convert to probabilities.

In [39]:
p = o / (o+1)
p

array([0.18242552, 0.40131234, 0.40131234, 0.9168273 ])

The likelihoods of the actual outcomes are $p$ where $y$ is 1 and $1-p$ where $y$ is 0. 

In [40]:
likes = np.where(y, p, 1-p)
likes

array([0.81757448, 0.40131234, 0.59868766, 0.9168273 ])

The likelihood of $y$ given $\beta$ is the product of `likes`:

In [41]:
like = np.prod(likes)
like

0.1800933529673034

Logistic regression works by searching for the values in $\beta$ that maximize `like`.

Here's an example using variables in the NSFG respondent file to predict whether a baby will be a boy or a girl.

In [42]:
live['boy'] = (live.babysex==1).astype(int)

The mother's age seems to have a small effect.

In [43]:
model = smf.logit('boy ~ agepreg', data=live)
results = model.fit()
summarize_results(results)

Optimization terminated successfully.
         Current function value: 0.693022
         Iterations 3
Intercept           : 0.0067    0.9457
agepreg             : 0.0010    0.7984
R^2      : 0.0000


The parameter of `agepreg` is positive, which suggests that older mothers are more likely to have boys, but the p-value is 0.798, which means that the apparent effect could easily be due to chance.

here’s a model that includes several factors believed to be associated with sex ratio:

In [69]:
formula = 'boy ~ agepreg + hpagelb + birthord + C(race)'
model = smf.logit(formula, data=live)
results = model.fit()
summarize_results(results)

Optimization terminated successfully.
         Current function value: 0.692916
         Iterations 3
Intercept                 : -0.0333    0.7504
C(race)[T.2]              : -0.0257    0.6163
C(race)[T.3]              : -0.0069    0.9348
agepreg                   : -0.0035    0.5288
hpagelb                   : 0.0052    0.2205
birthord                  : 0.0113    0.6102
R^2      : 0.0002


Along with mother’s age, this model includes father’s age at birth (`hpagelb`), birth order (`birthord`), and `race` as a categorical variable.

None of the estimated parameters are statistically significant. The pseudo-$R^2$ value is a little higher, but that could be due to chance.


## Accuracy

To make a prediction, we have to extract the exogenous and endogenous variables.

In [45]:
# dependent variable, or response variable
model.endog_names

'boy'

In [46]:
# predictors
model.exog_names

['Intercept', 'C(race)[T.2]', 'C(race)[T.3]', 'agepreg', 'hpagelb', 'birthord']

The baseline prediction strategy is to guess "boy".  In that case, we're right almost 51% of the time.

In [51]:
actual = model.endog
baseline = actual.mean()
r4(baseline)

0.5069

Since actual is encoded in binary integers, the mean is the fraction of boys, which is 0.507.

If we use the previous model, we can compute the number of predictions we get right.

In [52]:
predict = (results.predict() >= 0.5)
# multiply by actual yields 1 if we predict a boy and get it right, otherwise 0
true_pos = predict * actual
true_neg = (1 - predict) * (1 - actual)
sum(true_pos), sum(true_neg)

(3625.0, 832.0)

And the accuracy, which is slightly higher than the baseline.

In [53]:
acc = (sum(true_pos) + sum(true_neg)) / len(actual)
r4(acc)

0.5131

The result is 0.513, slightly better than the baseline, 0.507. But, you should not take this result too seriously. We used the same data to build and test the model, so the model may not have predictive power on new data.

To make a prediction for an individual, we have to get their information into a `DataFrame`.

In [54]:
columns = ['agepreg', 'hpagelb', 'birthord', 'race']
new = pd.DataFrame([[35, 39, 3, 2]], columns=columns)
y = results.predict(new)
y

0    0.513876
dtype: float64

This person has a 51% chance of having a boy (according to the model).

## Exercises

**Exercise:** Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

The following are the only variables I found that have a statistically significant effect on pregnancy length.

In [57]:
model = smf.ols('prglngth ~ birthord==1 + race==2 + nbrnaliv>1', data=live)
results = model.fit()
summarize_results(results)

Intercept                 : 38.7617    0.0000
birthord == 1[T.True]     : 0.1089    0.0071
race == 2[T.True]         : 0.1394    0.0010
nbrnaliv > 1[T.True]      : -1.5099    0.0000
R^2      : 0.0117
Std(ys)  : 1.9027
Std(res) : 1.8916


**Exercise:** The Trivers-Willard hypothesis suggests that for many mammals the sex ratio depends on “maternal condition”; that is, factors like the mother’s age, size, health, and social status. See https://en.wikipedia.org/wiki/Trivers-Willard_hypothesis

Some studies have shown this effect among humans, but results are mixed. In this chapter we tested some variables related to these factors, but didn’t find any with a statistically significant effect on sex ratio.

As an exercise, use a data mining approach to test the other variables in the pregnancy and respondent files. Can you find any factors with a substantial effect?

In [ ]:
join['boy'] = (join.babysex==1).astype(int)

In [64]:
def go_mining(df):
    """Searches for variables that predict birth weight.

    df: DataFrame of pregnancy records

    returns: list of (rsquared, variable name) pairs
    """
    
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula='boy ~ agepreg + ' + name
            model = smf.logit(formula, data=df);
            nobs = len(model.endog)
            if nobs < len(df)/2:
                continue
            results = model.fit();
            variables.append((results.prsquared, name))
        except:
            continue
    return variables

In [ ]:
# Solution

#Here are the 30 variables that yield the highest pseudo-R^2 values.

variables = go_mining(join)

In [74]:
variables.sort(reverse=True)
for rsq, name in variables[:30]:
    print(f'{name:20}: {rsq:0.5f}')

totalwgt_lb         : 0.00970
birthwgt_lb         : 0.00953
lbw1                : 0.00116
constat3            : 0.00105
fmarout5            : 0.00102
infever             : 0.00094
rmarout6            : 0.00094
nplaced             : 0.00084
pmarpreg            : 0.00082
coh1dur             : 0.00076
outcom02            : 0.00075
usefstp             : 0.00072
marout03            : 0.00067
agepreg_i           : 0.00065
splstwk1            : 0.00064
abort12             : 0.00064
rmarout03           : 0.00062
nummult34           : 0.00060
hlpmc               : 0.00059
mar1con1_i          : 0.00059
anymschp            : 0.00058
brnout_r            : 0.00057
brnout              : 0.00057
nummult41           : 0.00057
educmom             : 0.00056
fmarcon5            : 0.00056
numfirsm1           : 0.00055
agecon_i            : 0.00055
p1yhsage            : 0.00055
nummult17           : 0.00052


In [75]:
# Solution

# Eliminating variables that are not known during pregnancy and 
# others that are fishy for various reasons, here's the best model I could find:

formula='boy ~ agepreg + fmarout5==5 + infever==1'
model = smf.logit(formula, data=join)
results = model.fit()
summarize_results(results)

Optimization terminated successfully.
         Current function value: 0.691703
         Iterations 4
Intercept                 : -0.1936    0.1032
fmarout5 == 5[T.True]     : 0.1695    0.0006
infever == 1[T.True]      : 0.2369    0.0003
agepreg                   : 0.0053    0.2242
R^2      : 0.0019


**Exercise:** If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called `poisson`. It works the same way as `ols` and `logit`. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called `numbabes`.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [76]:
# Solution

# I used a nonlinear model of age.

join.numbabes.replace([97], np.nan, inplace=True)
join['age2'] = join.age_r**2

In [77]:
# Solution

formula='numbabes ~ age_r + age2 + age3 + C(race) + totincr + educat'
formula='numbabes ~ age_r + age2 + C(race) + totincr + educat'
model = smf.poisson(formula, data=join)
results = model.fit()
summarize_results(results)

Optimization terminated successfully.
         Current function value: 1.674288
         Iterations 7
Intercept                 : -1.0374    0.0000
C(race)[T.2]              : -0.1385    0.0000
C(race)[T.3]              : -0.1000    0.0001
age_r                     : 0.1557    0.0000
age2                      : -0.0020    0.0000
totincr                   : -0.0185    0.0000
educat                    : -0.0468    0.0000
R^2      : 0.0362


Now we can predict the number of children for a woman who is 35 years old, black, and a college
graduate whose annual household income exceeds $75,000

In [78]:
# Solution

columns = ['age_r', 'age2', 'age3', 'race', 'totincr', 'educat']
new = pd.DataFrame([[35, 35**2, 35**3, 1, 14, 16]], columns=columns)
results.predict(new)

0    2.496388
dtype: float64

**Exercise:** If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called `mnlogit`. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called `rmarital`.

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [88]:
name = 'stuff'
print(f'{name:26} :')

stuff                      :


In [94]:
# Solution

# Here's the best model I could find.

formula='rmarital ~ age_r + age2 + C(race) + totincr + educat'
model = smf.mnlogit(formula, data=join)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.079016
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8781
Model:                        MNLogit   Df Residuals:                     8746
Method:                           MLE   Df Model:                           30
Date:                Sat, 22 Jan 2022   Pseudo R-squ.:                  0.1699
Time:                        00:19:09   Log-Likelihood:                -9474.8
converged:                       True   LL-Null:                       -11414.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        9.0728      0.811     11.191      0.000       7.484      10.662
C(race)[T.2]    -0.9328      0.089    -10.447      0.000      -1.108      -0.758
C(race)[T.3]    -0.6251      0.138     -4.538      0.000      -0.895      -0.355
age_r           -0.3616      0.051     -7.065      0.000      -0.462      -0.261
age2             0.0048      0.001      5.993      0.000       0.003       0.006
totincr         -0.1290      0.012    -11.063      0.000      -0.152      -0.106
educat          -0.2020      0.019    -10.636      0.000      -0.239      -0.165
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        3.2123      3.025      1.062      0.288      -2.717       9.142
C(race)[T.2]    -0.3721      0.243     -1.534      0.125      -0.848       0.103
C(race)[T.3]     0.1692      0.341      0.496      0.620      -0.500       0.838
age_r           -0.3362      0.177     -1.900      0.057      -0.683       0.011
age2             0.0067      0.003      2.638      0.008       0.002       0.012
totincr         -0.3249      0.032    -10.060      0.000      -0.388      -0.262
educat          -0.1036      0.049     -2.113      0.035      -0.200      -0.008
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -3.4092      1.208     -2.822      0.005      -5.777      -1.041
C(race)[T.2]    -0.3069      0.094     -3.261      0.001      -0.491      -0.122
C(race)[T.3]    -0.8989      0.182     -4.951      0.000      -1.255      -0.543
age_r            0.1089      0.071      1.529      0.126      -0.031       0.248
age2            -0.0006      0.001     -0.616      0.538      -0.003       0.001
totincr         -0.2289      0.012    -19.522      0.000      -0.252      -0.206
educat           0.0677      0.017      3.998      0.000       0.035       0.101
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.8398      1.320     -2.152      0.031      -5.427      -0.253
C(race)[T.2]    -1.0473      0.105     -9.981      0.000      -1.253      -0.842
C(race)[T.3]    -0.5774      0.159     -3.641      0.000      -0.888      -0.267
age_r            0.2374      0.080      2.957      0.003       0.080       0.395
age2            -0.0035      0.001     -2.895      0.004      -0.006      -0.001
totincr         -0.2956      0.015    -20.012      0.000      -0.325      -0.267
educat          -0.0168      0.022     -0.771      0.441      -0.060       0.026
-

Make a prediction for a woman who is 25 years old, white, and a high
school graduate whose annual household income is about $45,000.

In [ ]:
# Solution

# This person has a 75% chance of being currently married, 
# a 13% chance of being "not married but living with opposite 
# sex partner", etc.

columns = ['age_r', 'age2', 'race', 'totincr', 'educat']
new = pd.DataFrame([[25, 25**2, 2, 11, 12]], columns=columns)
results.predict(new)